In [0]:
%python
import boto3
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# Retrieve the parameter values
job_id = dbutils.widgets.get("job_id")
usecase_id = dbutils.widgets.get("usecase_id")

# Print the parameter values
print(f"job_id: {job_id}")
print(f"usecase_id: {usecase_id}")

In [0]:
# get s3 params from metadata table 
s3_query = f"SELECT s3_bucket_name,s3_veeva_bucket_name,s3_STG_path,s3_veeva_path from com_us_alyt_ngebox.metadata_usecase where USECASE_ID = '{usecase_id}'"
s3_source_bucket_df = sqlContext.sql(s3_query).collect()[0]['s3_bucket_name']
s3_destination_bucket_df = sqlContext.sql(s3_query).collect()[0]['s3_veeva_bucket_name']
s3_source_path_df = sqlContext.sql(s3_query).collect()[0]['s3_STG_path']
s3_destination_path_df = sqlContext.sql(s3_query).collect()[0]['s3_veeva_path']

# assign the result dataframe to source and destination variable
source_bucket = s3_source_bucket_df
destination_bucket = s3_destination_bucket_df
source_path = s3_source_path_df
destination_path = s3_destination_path_df

# Create a Boto3 S3 client
s3_client = boto3.client('s3')
# List objects in the source path
response = s3_client.list_objects_v2(Bucket=source_bucket, Prefix=source_path)
# Copy files from the source path to the destination path
for obj in response['Contents']:
    key = obj['Key']
    destination_key = key.replace(source_path, destination_path)
    s3_client.copy_object(
        CopySource={'Bucket': source_bucket, 'Key': key},
        Bucket=destination_bucket,
        Key=destination_key
    )
    print(f"File {key} copied to {destination_bucket}/{destination_key}")

File NGE_Box/IN_N_OUT/OUT/RD/IENGAGE_PUBLISH/Takhzyro_Leads_Labs/STG/ copied to tpc-aws-ted-dev-iics-edpp-mount-usvga/amer_veeva/SrcFiles/iengage/data/current/
File NGE_Box/IN_N_OUT/OUT/RD/IENGAGE_PUBLISH/Takhzyro_Leads_Labs/STG/Veeva_Suggestion_File_Takhzyro_Leads_labs_20230623_105046.csv copied to tpc-aws-ted-dev-iics-edpp-mount-usvga/amer_veeva/SrcFiles/iengage/data/current/Veeva_Suggestion_File_Takhzyro_Leads_labs_20230623_105046.csv


In [0]:
class s3Push:
   def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": "S3 file succesfully loaded to the destination path"
      }
    }
    self.response = resp   

In [0]:
metadata_obj = s3Push()
metadata_obj.construct_microservice_response()
dbutils.notebook.exit(
  metadata_obj.response
)

{'nge_response': {'status': 200, 'body': 'S3 file succesfully loaded to the destination path'}}